In [4]:
import datetime
import json
import numpy as np
import requests
import pandas as pd
import streamlit as st
from copy import deepcopy

st.set_page_config(layout='wide', initial_sidebar_state='collapsed')

@st.cache(allow_output_mutation=True, suppress_st_warning=True)
def load_mapping():
    df = pd.read_csv("district_mapping.csv")
    return df

def filter_column(df, col, value):
    df_temp = deepcopy(df.loc[df[col] == value, :])
    return df_temp


mapping_df = load_mapping()

mapping_dict = pd.Series(mapping_df["district id"].values,
                         index = mapping_df["district name"].values).to_dict()

rename_mapping = {
    'date': 'Date',
    'min_age_limit': 'Minimum Age Limit',
    'available_capacity': 'Available Capacity',
    'pincode': 'Pincode',
    'name': 'Hospital Name',
    'state_name' : 'State',
    'district_name' : 'District',
    'block_name': 'Block Name',
    'fee_type' : 'Fees'
    }

st.title('CoWIN Vaccination Slot Availability')
st.info('The CoWIN APIs are geo-fenced so sometimes you may not see an output! Please try after sometime ')

# numdays = st.sidebar.slider('Select Date Range', 0, 100, 10)
unique_districts = list(mapping_df["district name"].unique())
unique_districts.sort()

left_column_1, right_column_1 = st.beta_columns(2)
with left_column_1:
    numdays = st.slider('Select Date Range', 0, 100, 5)

with right_column_1:
    dist_inp = st.selectbox('Select District', unique_districts)

DIST_ID = mapping_dict[dist_inp]

base = datetime.datetime.today()
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
date_str = [x.strftime("%d-%m-%Y") for x in date_list]

final_df = None
for INP_DATE in date_str:
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, INP_DATE)
    response = requests.get(URL)
    if (response.ok) and ('centers' in json.loads(response.text)):
        resp_json = json.loads(response.text)['centers']
        if resp_json is not None:
            df = pd.DataFrame(resp_json)
            if len(df):
                df = df.explode("sessions")
                df['min_age_limit'] = df.sessions.apply(lambda x: x['min_age_limit'])
                df['available_capacity'] = df.sessions.apply(lambda x: x['available_capacity'])
                df['date'] = df.sessions.apply(lambda x: x['date'])
                df = df[["date", "available_capacity", "min_age_limit", "pincode", "name", "state_name", "district_name", "block_name", "fee_type"]]
                if final_df is not None:
                    final_df = pd.concat([final_df, df])
                else:
                    final_df = deepcopy(df)
            else:
                st.error("No rows in the data Extracted from the API")
#     else:
#         st.error("Invalid response")

if (final_df is not None) and (len(final_df)):
    final_df.drop_duplicates(inplace=True)
    final_df.rename(columns=rename_mapping, inplace=True)

    left_column_2, center_column_2, right_column_2 = st.beta_columns(3)
    with left_column_2:
        valid_pincodes = list(np.unique(final_df["Pincode"].values))
        pincode_inp = st.selectbox('Select Pincode', [""] + valid_pincodes)
        if pincode_inp != "":
            final_df = filter_column(final_df, "Pincode", pincode_inp)

    with center_column_2:
        valid_age = [18, 45]
        age_inp = st.selectbox('Select Minimum Age', [""] + valid_age)
        if age_inp != "":
            final_df = filter_column(final_df, "Minimum Age Limit", age_inp)

    with right_column_2:
        valid_payments = ["Free", "Paid"]
        pay_inp = st.selectbox('Select Free or Paid', [""] + valid_payments)
        if pay_inp != "":
            final_df = filter_column(final_df, "Fees", pay_inp)

    table = deepcopy(final_df)
    table.reset_index(inplace=True, drop=True)
    st.table(table)
else:
    st.error("Unable to fetch data currently, please try after sometime")

st.markdown("_- Chandan kumar_")


ModuleNotFoundError: No module named 'streamlit'

In [6]:
pip install streamlit

  Using cached base58-2.1.0-py3-none-any.whl (5.6 kB)



  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\mysore\anaconda3\python.exe' 'C:\Users\mysore\AppData\Local\Temp\pip-standalone-pip-hjwm9c90\__env_pip__.zip\pip' install --ignore-installed --no-user --prefix 'C:\Users\mysore\AppData\Local\Temp\pip-build-env-nkb9mknr\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- 'cython >= 0.29' 'numpy==1.16.6; python_version<'"'"'3.9'"'"'' 'numpy==1.19.4; python_version>='"'"'3.9'"'"'' setuptools setuptools_scm wheel
       cwd: None
  Complete output (578 lines):
  Ignoring numpy: markers 'python_version >= "3.9"' don't match your environment
    Using cached setuptools_scm-6.0.1-py3-none-any.whl (27 kB)
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\mysore\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\mysore\\AppData\\Local\\Temp\\pip-install-_kl_wzp1\\numpy_43414aa354404f8784bbb1d4f4d9f7

  Using cached pyarrow-4.0.0.tar.gz (710 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached pyarrow-3.0.0.tar.gz (682 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached pyarrow-2.0.0.tar.gz (58.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached pyarrow-1.0.1.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached pyarrow-1.0.0.tar.gz (1.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached pyarrow-0.17.1.tar.gz (2.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached pyarrow-0.17.0.tar.gz (3.7 MB)
  Installing build dependencies: started
  Installing build depe


    customize PGroupFlangCompiler
    Could not locate executable flang
    don't know how to compile Fortran code on platform 'nt'
      NOT AVAILABLE
  
    atlas_3_10_blas_threads_info:
    Setting PTATLAS=ATLAS
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries tatlas not found in ['C:\\Users\\mysore\\anaconda3\\lib', 'C:\\', 'C:\\Users\\mysore\\anaconda3\\libs']
      NOT AVAILABLE
  
    atlas_3_10_blas_info:
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries satlas not found in ['C:\\Users\\mysore\\anaconda3\\lib', 'C:\\', 'C:\\Users\\mysore\\anaconda3\\libs']
      NOT AVAILABLE
  
    atlas_blas_threads_info:
    Setting PTATLAS=ATLAS
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries ptf77blas,ptcblas,atlas not found in [


  Using cached protobuf-3.15.8-py2.py3-none-any.whl (173 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
Failed to build pyarrow


...

      self.calc_info()
      NOT AVAILABLE
  
    C:\Users\mysore\anaconda3\lib\distutils\dist.py:274: UserWarning: Unknown distribution option: 'define_macros'
      warnings.warn(msg)
    running bdist_wheel
    running build
    running config_cc
    unifing config_cc, config, build_clib, build_ext, build commands --compiler options
    running config_fc
    unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
    running build_src
    build_src
    building py_modules sources
    creating build
    creating build\src.win32-3.8
    creating build\src.win32-3.8\numpy
    creating build\src.win32-3.8\numpy\distutils
    building library "npymath" sources
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
    ----------------------------------------
    

      libraries f77blas,cblas,atlas not found in C:\Users\mysore\anaconda3\lib
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries lapack_atlas not found in C:\
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries f77blas,cblas,atlas not found in C:\
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries lapack_atlas not found in C:\Users\mysore\anaconda3\libs
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries f77blas,cblas,atlas not found in C:\Users\mysore\anaconda3\libs
    <class 'numpy.distutils.system_info.atlas_info'>
      NOT AVAILABLE
  
    lapack_info:
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutil

      customize MSVCCompiler
        libraries blas not found in ['C:\\Users\\mysore\\anaconda3\\lib', 'C:\\', 'C:\\Users\\mysore\\anaconda3\\libs']
        NOT AVAILABLE
  
      C:\Users\mysore\AppData\Local\Temp\pip-install-1nf808d2\numpy_99921128fd95486da0bae5567fe24373\numpy\distutils\system_info.py:625: UserWarning:
          Blas (http://www.netlib.org/blas/) libraries not found.
          Directories to search for the libraries can be specified in the
          numpy/distutils/site.cfg file (section [blas]) or by setting
          the BLAS environment variable.
        self.calc_info()
      blas_src_info:
        NOT AVAILABLE
  
      C:\Users\mysore\AppData\Local\Temp\pip-install-1nf808d2\numpy_99921128fd95486da0bae5567fe24373\numpy\distutils\system_info.py:625: UserWarning:
          Blas (http://www.netlib.org/blas/) sources not found.
          Directories to search for the sources can be specified in the
          numpy/distutils/site.cfg file (section [blas_src]) or by 

      running install
      running build
      running config_cc
      unifing config_cc, config, build_clib, build_ext, build commands --compiler options
      running config_fc
      unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
      running build_src
      build_src
      building py_modules sources
      building library "npymath" sources
      No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      ----------------------------------------
  ERROR: Command errored out with exit status 1: 'C:\Users\mysore\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\mysore\\AppData\\Local\\Temp\\pip-install-1nf808d2\\numpy_99921128fd95486da0bae5567fe24373\\setup.py'"'"'; __file__='"'"'C:\\Users\\mysore\\A

  copying pyarrow\include\arrow\io\interfaces.h -> build\lib.win32-3.8\pyarrow\include\arrow\io
  copying pyarrow\include\arrow\io\memory.h -> build\lib.win32-3.8\pyarrow\include\arrow\io
  copying pyarrow\include\arrow\io\mman.h -> build\lib.win32-3.8\pyarrow\include\arrow\io
  copying pyarrow\include\arrow\io\readahead.h -> build\lib.win32-3.8\pyarrow\include\arrow\io
  copying pyarrow\include\arrow\io\slow.h -> build\lib.win32-3.8\pyarrow\include\arrow\io
  copying pyarrow\include\arrow\io\test_common.h -> build\lib.win32-3.8\pyarrow\include\arrow\io
  creating build\lib.win32-3.8\pyarrow\include\arrow\ipc
  copying pyarrow\include\arrow\ipc\api.h -> build\lib.win32-3.8\pyarrow\include\arrow\ipc
  copying pyarrow\include\arrow\ipc\dictionary.h -> build\lib.win32-3.8\pyarrow\include\arrow\ipc
  copying pyarrow\include\arrow\ipc\feather.h -> build\lib.win32-3.8\pyarrow\include\arrow\ipc
  copying pyarrow\include\arrow\ipc\json_integration.h -> build\lib.win32-3.8\pyarrow\include\arrow